<a href="https://colab.research.google.com/github/RohitGanji/white-blood-cancer-all-mm/blob/main/White_Blood_Cancer_B-ALL/MM_Classification_using_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# White Blood Cancer B-ALL/MM Classification using Transfer Learning

You can download the dataset from [here](https://drive.google.com/drive/folders/18p96I52S1CyabBeAE8g5vcfhERh8ZJTi?usp=sharing).

## Creating Dataset from Google Drive

In [2]:
import tensorflow as tf

train_dir = '/content/drive/MyDrive/Datasets/SN-AM-BALL-MM/training'
test_dir = '/content/drive/MyDrive/Datasets/SN-AM-BALL-MM/testing'

IMG_SIZE = (224, 224)

train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 label_mode="binary",
                                                                 image_size=IMG_SIZE,
                                                                 validation_split=0.2,
                                                                 subset='training',
                                                                 seed=1302)

val_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 label_mode="binary",
                                                                 image_size=IMG_SIZE,
                                                                 validation_split=0.2,
                                                                 subset='validation',
                                                                 seed=1302)

test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                 label_mode="binary",
                                                                 image_size=IMG_SIZE)

Found 173 files belonging to 2 classes.
Using 139 files for training.
Found 173 files belonging to 2 classes.
Using 34 files for validation.
Found 29 files belonging to 2 classes.


## Data Augmentation

In [3]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

# Setup data augmentation
data_augmentation = Sequential([
  preprocessing.RandomFlip("horizontal"),
  preprocessing.RandomRotation(0.2),
  preprocessing.RandomHeight(0.2),
  preprocessing.RandomWidth(0.2),
  preprocessing.RandomZoom(0.2),
], name = "data_augmentation")

## F1 Metric and Callbacks

In [4]:
import keras.backend as K

def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [18]:
# Create checkpoint callback
checkpoint_path = "/content/model"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True,
                                                         monitor="val_f1_metric",
                                                         save_best_only=True)

## Model

In [19]:
# Create a base model
input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB3(include_top=False)
base_model.trainable = True

for layer in base_model.layers[:-3]:
  layer.trainable = False

# Setup model architecture
inputs = layers.Input(shape=(224,224,3), name="input_shape")
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D(name="GAP")(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid", name="output_layer")(x)
model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy", f1_metric])

In [20]:
# Fit the model
history = model.fit(train_data,
                    epochs=30,
                    steps_per_epoch=int(len(train_data)),
                    validation_data=val_data,
                    validation_steps=int(len(val_data)),
                    callbacks=[checkpoint_callback])

Epoch 1/30
5/5 [==============================] - 24s 2s/step - loss: 0.5216 - accuracy: 0.7338 - f1_metric: 0.8161 - val_loss: 0.4001 - val_accuracy: 0.7647 - val_f1_metric: 0.8889
Epoch 2/30
5/5 [==============================] - 9s 804ms/step - loss: 0.0890 - accuracy: 1.0000 - f1_metric: 1.0000 - val_loss: 0.8489 - val_accuracy: 0.5588 - val_f1_metric: 0.6742
Epoch 3/30
5/5 [==============================] - 9s 781ms/step - loss: 0.0553 - accuracy: 0.9784 - f1_metric: 0.9838 - val_loss: 0.4455 - val_accuracy: 0.7941 - val_f1_metric: 0.9054
Epoch 4/30
5/5 [==============================] - 10s 1s/step - loss: 0.0275 - accuracy: 0.9856 - f1_metric: 0.9897 - val_loss: 0.2682 - val_accuracy: 0.9118 - val_f1_metric: 0.9545
Epoch 5/30
5/5 [==============================] - 9s 860ms/step - loss: 0.0170 - accuracy: 0.9928 - f1_metric: 0.9778 - val_loss: 0.4127 - val_accuracy: 0.8235 - val_f1_metric: 0.9118
Epoch 6/30
5/5 [==============================] - 10s 955ms/step - loss: 0.0019 - ac

In [23]:
model.load_weights("/content/model")